# INDIVIDUAL ASSIGNMENT SPARK - FRANCISCO VIGO



### Importing Libraries

In [2]:
import findspark
import pandas as pd
findspark.init()

from IPython.display import display, Markdown
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import max, min, avg, stddev, round
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import when, count, col, countDistinct, desc, first, lit



sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

### Reading my data

In [3]:
carsDF = spark.read .option("inferSchema", "true").option("header", "true").csv("spark_cars_final_dataset2.csv")

In [4]:
carsDF.printSchema()

nrows = carsDF.count() #Number of Rows
ncols = len(carsDF.columns)

print("My data has", nrows ,"rows and", ncols , "features.-")

#My dataset has 12 features and 150.000 rows.

root
 |-- region: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- manufacturer: string (nullable = true)
 |-- model: string (nullable = true)
 |-- condition: string (nullable = true)
 |-- cylinders: string (nullable = true)
 |-- fuel: string (nullable = true)
 |-- odometer: integer (nullable = true)
 |-- transmission: string (nullable = true)
 |-- size: string (nullable = true)
 |-- paint_color: string (nullable = true)

My data has 200000 rows and 12 features.-


In [5]:
carsDF.cache()
'''
Im requesting Spark to keep my dataset in memory, because I'm going to use it a lot.
.cache is very similar to .persist()
'''

#Some very basic data exploration.
#I detect that I have nulls in the feature "paint_color" and in manufacturer.

carsDF.select('year','manufacturer','model','price','odometer','paint_color').show()

+----+-------------+--------------------+-----+--------+-----------+
|year| manufacturer|               model|price|odometer|paint_color|
+----+-------------+--------------------+-----+--------+-----------+
|2016|         jeep|wrangler willys w...|29200|   15800|      white|
|2007|        dodge|             caliber| 3600|  138153|      black|
|2013|   volkswagen|              passat|13999|   14145|     silver|
|2013|   volkswagen|            jetta se| 4675|   90000|      white|
|1994|         ford|   thunderbird coupe| 7900|   99000|        red|
|2018|       nissan|              altima|14000|   55650|      black|
|2012|         ford|         f-350sd xlt|29990|   94939|       null|
|2013|        honda|               civic| 7800|   99800|      brown|
|2009|       toyota|              sienna| 5995|  131641|     custom|
|2013|     cadillac|                 srx|10950|  109278|       grey|
|2011|       toyota|           camry xle| 7995|  124695|       blue|
|2016|   volkswagen|              

In [6]:
import pyspark.sql.functions as F

mycolumns = carsDF.columns

#I use the for loop to check how many Nulls I have in each row.
#As I discovered during my data exploration, I have a lot of Nulls in the columns Condition, 
#Paint_color, Manufacturer and Size.


print("Checking for nulls:")
carsDF.select([count(when(F.col(d).isNull(), d)).alias(d) for d in mycolumns]).show()

#Since I'm new to Spark, I crossvalidate if my function is working correctly....

print("Number of Nulls in the Column Manufacturer =",carsDF\
    .where(F.col('manufacturer').isNull())\
    .select('manufacturer').count()) 

#Both numbers match.

#I have to drop my Nulls to continue with my analisis. 





Checking for nulls:
+------+-----+----+------------+-----+---------+---------+----+--------+------------+------+-----------+
|region|price|year|manufacturer|model|condition|cylinders|fuel|odometer|transmission|  size|paint_color|
+------+-----+----+------------+-----+---------+---------+----+--------+------------+------+-----------+
|     0|    0|   0|        6746| 1620|    55853|        0|1833|       0|          57|101491|      34298|
+------+-----+----+------------+-----+---------+---------+----+--------+------------+------+-----------+

Number of Nulls in the Column Manufacturer = 6746


In [7]:
carsDF2 = carsDF.na.drop(how='any')

carsDF2.select('year','manufacturer','model','price','odometer','paint_color').show()

nrows2 = carsDF2.count() #Number of Rows
ncols2 = len(carsDF2.columns)


#I check for nulls again to make sure that my .na.drop worked. 


print("Checking for nulls:")
carsDF2.select([count(when(F.col(d).isNull(), d)).alias(d) for d in mycolumns]).show()

print("My cleaned data has", nrows2 ,"rows and", ncols2 , "features.-")
print("I have dropped", carsDF.count()-carsDF2.count(), "rows")






+----+-------------+--------------------+-----+--------+-----------+
|year| manufacturer|               model|price|odometer|paint_color|
+----+-------------+--------------------+-----+--------+-----------+
|2013|   volkswagen|            jetta se| 4675|   90000|      white|
|1994|         ford|   thunderbird coupe| 7900|   99000|        red|
|2018|       nissan|              altima|14000|   55650|      black|
|2013|        honda|               civic| 7800|   99800|      brown|
|2009|       toyota|              sienna| 5995|  131641|     custom|
|2013|     cadillac|                 srx|10950|  109278|       grey|
|2007|          ram|        2500 cummins|22000|  110000|        red|
|2016|mercedes-benz|             s-class|    0|   33252|      white|
|2008|        acura|                 mdx|    0|  110190|     silver|
|2004|         ford|     f350 super duty|12000|  130000|       grey|
|2017|      hyundai|           accent se|13990|   47864|      white|
|2003|    chevrolet|         trail

In [8]:
#I caché my new cleaned dataset to keep it in memory.
carsDF2.cache()

carsDF2.printSchema()

root
 |-- region: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- manufacturer: string (nullable = true)
 |-- model: string (nullable = true)
 |-- condition: string (nullable = true)
 |-- cylinders: string (nullable = true)
 |-- fuel: string (nullable = true)
 |-- odometer: integer (nullable = true)
 |-- transmission: string (nullable = true)
 |-- size: string (nullable = true)
 |-- paint_color: string (nullable = true)



In [9]:
#Computing Statistics for my numerical variables

carsDF2.select('year','price','odometer').summary().show()

###
#I have very old cars (1915) but in average the used cars that are for sale are 12 years old, because
#the mean is around 2008.
#For the year, the mean (2008) is smaller than the median (50% percentil, 2010) --> the distribution is negatively skewed.
#In average, the used cars for sale cost 24.330 USD and they have around 117.585 miles.
###


+-------+-----------------+-----------------+------------------+
|summary|             year|            price|          odometer|
+-------+-----------------+-----------------+------------------+
|  count|            81760|            81760|             81760|
|   mean|2008.861350293542| 24330.1167074364| 117585.3887353229|
| stddev|7.595095067547672|2946779.863707078|140058.86616449116|
|    min|             1915|                0|                 0|
|    25%|             2006|             3995|             69078|
|    50%|             2010|             7995|            111521|
|    75%|             2014|            14995|            153534|
|    max|             2020|        831365932|          10000000|
+-------+-----------------+-----------------+------------------+



In [10]:
#Let's check how many brands, models and colors do I have in my dataset.


#Using the for lop
carsDF2.select([countDistinct(e).alias(e) for e in ['manufacturer','model','transmission','paint_color']]).show()

#Using another method

carsDF2.select(countDistinct('manufacturer').alias('Manufacturer'),\
               countDistinct('model').alias('Model'),\
               countDistinct('transmission').alias('Trans.'),\
               countDistinct('paint_color').alias('Color'))\
               .show()

# I have:
# 43 different brands
# 9846 different models
# 3 types of transmission
# 12 different colors


+------------+-----+------------+-----------+
|manufacturer|model|transmission|paint_color|
+------------+-----+------------+-----------+
|          43| 9846|           3|         12|
+------------+-----+------------+-----------+

+------------+-----+------+-----+
|Manufacturer|Model|Trans.|Color|
+------------+-----+------+-----+
|          43| 9846|     3|   12|
+------------+-----+------+-----+



In [11]:
# I will create some factors for the variable Odometer using its Statistics.
# [0 - 66000] -> Low Miles
# [66000 - 152147] -> Mid Miles
# [152147 - 10000000] -> High Miles

carsDF3 = carsDF2\
   .withColumn("odometer",     when((col("odometer")>=0) & (col("odometer")<=69078),"Low Miles")\
                               .when((col("odometer")>69078) & (col("odometer")<=153534),"Mid Miles")\
                               .otherwise("High Miles"))\
   .withColumn("year",     when((col("year")>=0) & (col("year")<=1969),"Antique")\
                               .when((col("year")> 1970) & (col("year")<=1979),"70's")\
                               .when((col("year")> 1979) & (col("year")<=1989),"80's")\
                               .when((col("year")> 1989) & (col("year")<=1999),"90's")\
                               .when((col("year")> 1999) & (col("year")<=2009),"00's")\
                               .when((col("year")== 2010),2010)\
                               .when((col("year")== 2011),2011)\
                               .when((col("year")== 2012),2012)\
                               .when((col("year")== 2013),2013)\
                               .when((col("year")== 2014),2014)\
                               .when((col("year")== 2015),2015)\
                               .when((col("year")== 2016),2016)\
                               .when((col("year")== 2017),2017)\
                               .when((col("year")== 2018),2018)\
                               .when((col("year")== 2019),2019)\
                               .when((col("year")== 2020),2020)\
                               .otherwise("Other"))



carsDF3.select('year','manufacturer','model','price','odometer','paint_color').show()

carsDF3.cache()


+----+-------------+--------------------+-----+----------+-----------+
|year| manufacturer|               model|price|  odometer|paint_color|
+----+-------------+--------------------+-----+----------+-----------+
|2013|   volkswagen|            jetta se| 4675| Mid Miles|      white|
|90's|         ford|   thunderbird coupe| 7900| Mid Miles|        red|
|2018|       nissan|              altima|14000| Low Miles|      black|
|2013|        honda|               civic| 7800| Mid Miles|      brown|
|00's|       toyota|              sienna| 5995| Mid Miles|     custom|
|2013|     cadillac|                 srx|10950| Mid Miles|       grey|
|00's|          ram|        2500 cummins|22000| Mid Miles|        red|
|2016|mercedes-benz|             s-class|    0| Low Miles|      white|
|00's|        acura|                 mdx|    0| Mid Miles|     silver|
|00's|         ford|     f350 super duty|12000| Mid Miles|       grey|
|2017|      hyundai|           accent se|13990| Low Miles|      white|
|00's|

DataFrame[region: string, price: int, year: string, manufacturer: string, model: string, condition: string, cylinders: string, fuel: string, odometer: string, transmission: string, size: string, paint_color: string]

In [12]:
#Is there a connection between the Odometer and Price?

from pyspark.sql.functions import max, min, avg, stddev, round
from pyspark.sql.types import IntegerType

print("Is there a connection between the Odometer and Price?")

carsDF3.groupBy("odometer")\
              .agg(round(avg("price"),0).alias("Price")).orderBy("price",ascending=False).show()

print("As expected, the average price of used cars decreases in cars of higher odometer \n\n")

#Is there a connection between Year and Price?

print("Is there a connection between the Year and Price? \n")

carsDF3.groupBy("year")\
              .agg(round(avg("price"),0).alias("Price"),count("price").alias("Count"))\
              .orderBy("year",ascending=True).show()

print("We can not say that there is a connection between Price and Year because we have some exceptions.")
print("For example, cars from 2013 are cheaper than cars from 2012.")
print("Another curious thing is that cars from 2019 are cheaper in average than cars from 2018.")
print("This may be related with the number of observations for each year.")
print("Another intresting thing is that cars from the 70's are more expensive than those from the 80's and 90's. This may be related with the revaluation of classic cars.")
      
      

Is there a connection between the Odometer and Price?
+----------+-------+
|  odometer|  Price|
+----------+-------+
| Low Miles|59360.0|
| Mid Miles|15735.0|
|High Miles| 6503.0|
+----------+-------+

As expected, the average price of used cars decreases in cars of higher odometer 


Is there a connection between the Year and Price? 

+-------+--------+-----+
|   year|   Price|Count|
+-------+--------+-----+
|   00's| 31868.0|32835|
|   2010|  8842.0| 4442|
|   2011| 11547.0| 5404|
|   2012| 17999.0| 5661|
|   2013| 12961.0| 5645|
|   2014| 19452.0| 5247|
|   2015| 21936.0| 5224|
|   2016| 18132.0| 4810|
|   2017| 35665.0| 3404|
|   2018| 22097.0| 2176|
|   2019| 20777.0| 1355|
|   2020| 35295.0|   70|
|   70's| 10714.0|  403|
|   80's|  6877.0|  684|
|   90's|  4897.0| 3885|
|Antique|278737.0|  478|
|  Other| 22418.0|   37|
+-------+--------+-----+

We can not say that there is a connection between Price and Year because we have some exceptions.
For example, cars from 2013 are cheape

In [13]:
carsDF3.groupBy("year")\
              .agg(round(avg("price"),0).alias("Price"),\
                   count("price").alias("Count"),\
                   round(max("price"),0).alias("Max Prince")).orderBy("year",ascending=True).show()

print("As suspected, both prices for 2020, 2019 and 2018 have less information than the previous years and therefore they are not relevant for our analisis.-")

+-------+--------+-----+----------+
|   year|   Price|Count|Max Prince|
+-------+--------+-----+----------+
|   00's| 31868.0|32835| 831365932|
|   2010|  8842.0| 4442|    149995|
|   2011| 11547.0| 5404|   1950000|
|   2012| 17999.0| 5661|  18880499|
|   2013| 12961.0| 5645|    150000|
|   2014| 19452.0| 5247|  26880499|
|   2015| 21936.0| 5224|  23880499|
|   2016| 18132.0| 4810|    295000|
|   2017| 35665.0| 3404|  36980499|
|   2018| 22097.0| 2176|     77500|
|   2019| 20777.0| 1355|     85500|
|   2020| 35295.0|   70|     99999|
|   70's| 10714.0|  403|    123456|
|   80's|  6877.0|  684|     68500|
|   90's|  4897.0| 3885|    500000|
|Antique|278737.0|  478| 123456789|
|  Other| 22418.0|   37|     68500|
+-------+--------+-----+----------+

As suspected, both prices for 2020, 2019 and 2018 have less information than the previous years and therefore they are not relevant for our analisis.-


In [14]:
#Average price per manufacturer?

carsDF3.groupBy("manufacturer")\
                .agg(round(avg("price"),0).alias("Avg Price"),\
                   round(max("price"),0).alias("Max Prince"), \
                  count('manufacturer').alias("Count"))\
                .orderBy("Avg Price",ascending=True).show(200)

+---------------+---------+----------+-----+
|   manufacturer|Avg Price|Max Prince|Count|
+---------------+---------+----------+-----+
|         saturn|   3588.0|     80000|  337|
|        mercury|   4119.0|     85000|  415|
|       chrysler|   6497.0|     69995| 1536|
|          volvo|   6503.0|     44500|  568|
|          mazda|   6558.0|     89550|  980|
|          acura|   7154.0|     46800|  746|
|        hyundai|   7236.0|    135000| 1862|
|            kia|   7284.0|     53785| 1370|
|          buick|   7326.0|    124900| 1020|
|           fiat|   7435.0|     17995|  114|
|     volkswagen|   7435.0|     80000| 1791|
|           mini|   7746.0|     70000|  355|
|     mitsubishi|   7860.0|     75000|  473|
|         nissan|   8206.0|     89500| 4294|
|          dodge|   8539.0|     76525| 3069|
|         subaru|   8837.0|     39000| 1720|
|     alfa-romeo|   8892.0|     54500|   25|
|       infiniti|   9244.0|     54995|  592|
|         datsun|   9905.0|     20000|   11|
|         

In [15]:
#Average price per manufacturer? Keep only significant manufacturers (n>300)


carsDF3.groupBy("manufacturer")\
                .agg(round(avg("price"),0).alias("Avg Price"),\
                   round(max("price"),0).alias("Max Prince"), \
                  count('manufacturer').alias("Count"))\
                .where(count('manufacturer') > 300)\
                .orderBy("Avg Price",ascending=True).show(200)




+-------------+---------+----------+-----+
| manufacturer|Avg Price|Max Prince|Count|
+-------------+---------+----------+-----+
|       saturn|   3588.0|     80000|  337|
|      mercury|   4119.0|     85000|  415|
|     chrysler|   6497.0|     69995| 1536|
|        volvo|   6503.0|     44500|  568|
|        mazda|   6558.0|     89550|  980|
|        acura|   7154.0|     46800|  746|
|      hyundai|   7236.0|    135000| 1862|
|          kia|   7284.0|     53785| 1370|
|        buick|   7326.0|    124900| 1020|
|   volkswagen|   7435.0|     80000| 1791|
|         mini|   7746.0|     70000|  355|
|   mitsubishi|   7860.0|     75000|  473|
|       nissan|   8206.0|     89500| 4294|
|        dodge|   8539.0|     76525| 3069|
|       subaru|   8837.0|     39000| 1720|
|     infiniti|   9244.0|     54995|  592|
|        lexus|  10007.0|    150000| 1041|
|          bmw|  10068.0|     99999| 2161|
|      lincoln|  10156.0|   1150000|  610|
|        honda|  10800.0|  15550499| 4674|
|         a

In [16]:
# Count of distinct 

for columnName in carsDF3.columns:
    distinctValues = carsDF3.select(columnName)\
                            .distinct()\
                            .count()
    
    # Don't forget to indent this line to indicate it is also inside the loop
    print("There are", distinctValues, "distinct values in column", columnName)



There are 401 distinct values in column region
There are 4624 distinct values in column price
There are 17 distinct values in column year
There are 43 distinct values in column manufacturer
There are 9846 distinct values in column model
There are 6 distinct values in column condition
There are 8 distinct values in column cylinders
There are 5 distinct values in column fuel
There are 3 distinct values in column odometer
There are 3 distinct values in column transmission
There are 4 distinct values in column size
There are 12 distinct values in column paint_color


In [17]:
#5 dinstict values in fuel? Let's do some data exploration to find out whats going on.

carsDF3.groupBy("fuel")\
              .agg(round(avg("price"),0).alias("Avg Price"),count("size").alias("N."), round((count("size")/carsDF3.count()*100),1).alias("% of total"))\
              .orderBy("Avg Price",ascending=True).show()

print("Gas fueled cars are by far the most popular, because they represent 91.5% of our total dataset.-")
print("Hybrid cars (0.9%) are 9 times more popular than electric cars (0.1%).-")

+--------+---------+-----+----------+
|    fuel|Avg Price|   N.|% of total|
+--------+---------+-----+----------+
|  hybrid|   8994.0|  767|       0.9|
|   other|  10418.0|   96|       0.1|
|electric|  18975.0|   51|       0.1|
|  diesel|  22106.0| 6015|       7.4|
|     gas|  24688.0|74831|      91.5|
+--------+---------+-----+----------+

Gas fueled cars are by far the most popular, because they represent 91.5% of our total dataset.-
Hybrid cars (0.9%) are 9 times more popular than electric cars (0.1%).-


In [18]:
# Are bigger cars more expensive than smaller ones?


carsDF3.groupBy("size")\
              .agg(round(avg("price"),0).alias("Avg Price"),count("size").alias("N."), round((count("size")/carsDF3.count()*100),1).alias("% of total"))\
              .orderBy("Avg Price",ascending=True).show()

print("Full size are more expensive than mid-size, but both sub-compact and compact are on the top of the list. ")


+-----------+---------+-----+----------+
|       size|Avg Price|   N.|% of total|
+-----------+---------+-----+----------+
|   mid-size|   8709.0|24190|      29.6|
|  full-size|  17800.0|46339|      56.7|
|sub-compact|  28087.0| 1231|       1.5|
|    compact|  91914.0|10000|      12.2|
+-----------+---------+-----+----------+

Full size are more expensive than mid-size, but both sub-compact and compact are on the top of the list. 


In [19]:
# Are automatic cars more expensive in average than manual cars?

carsDF3.groupBy("transmission")\
              .agg(round(avg("price"),0).alias("Avg Price"),count("size").alias("N."), round((count("size")/carsDF3.count()*100),1).alias("% of total"))\
              .orderBy("Avg Price",ascending=True).show()


print("Manual cars are in average cheaper than automatic ones")
print("92.4% of the cars in our dataset are automatic")

+------------+---------+-----+----------+
|transmission|Avg Price|   N.|% of total|
+------------+---------+-----+----------+
|       other|   7960.0|  759|       0.9|
|      manual|   9341.0| 5427|       6.6|
|   automatic|  25571.0|75574|      92.4|
+------------+---------+-----+----------+

Manual cars are in average cheaper than automatic ones
92.4% of the cars in our dataset are automatic


In [20]:
# Price per color? Is there a color that nobody wants and therefore it price is very low?

carsDF3.groupBy("paint_color")\
              .agg(round(avg("price"),0).alias("Avg Price"),count("size").alias("N."), round((count("size")/carsDF3.count()*100),1).alias("% of total"))\
              .orderBy("Avg Price",ascending=True).show()

print("White, black, grey and silver are the more popular colors.")

+-----------+---------+-----+----------+
|paint_color|Avg Price|   N.|% of total|
+-----------+---------+-----+----------+
|      green|   7748.0| 2602|       3.2|
|     purple|   8493.0|  252|       0.3|
|      brown|   8659.0| 2351|       2.9|
|     custom|  10545.0| 2156|       2.6|
|       grey|  10597.0| 9598|      11.7|
|     orange|  12557.0|  450|       0.6|
|     yellow|  12737.0|  559|       0.7|
|      black|  13260.0|14864|      18.2|
|        red|  13433.0| 8185|      10.0|
|      white|  17770.0|20212|      24.7|
|       blue|  23800.0| 8392|      10.3|
|     silver|  77717.0|12139|      14.8|
+-----------+---------+-----+----------+

White, black, grey and silver are the more popular colors.


### RECAP
##### We are going to focus in:

Automcatic.
Gas fueled.
White, black, silver or gray.

For our analysis we are going to exclude antique and old cars, because they are not relevant for our business case.
We will exclude cars older than 2014.
We exclude full-size cars because we don't want a truck.

#### Model Hunt


In [21]:
targetDF = carsDF3\
   .where(col("transmission") == "automatic" )\
   .where((col("paint_color") == "black") |\
          (col("paint_color") == "silver") |\
          (col("paint_color") == "white") |\
          (col("paint_color") == "grey"))\
    .where(col("fuel") == "gas")\
    .where(col("year") > 2014)\
    .where(col("size") != "full-size")

print("Our target dataset has", targetDF.count(), "rows.-")

Our target dataset has 5122 rows.-


In [22]:
targetDF.groupBy("manufacturer")\
              .agg(round(avg("price"),0).alias("Avg Price"),\
                   count('manufacturer').alias("N."), round((count("manufacturer")/targetDF.count()*100),1).alias("% of total"))\
                 .orderBy("% of total",ascending=False).show(200)



+-------------+---------+---+----------+
| manufacturer|Avg Price| N.|% of total|
+-------------+---------+---+----------+
|    chevrolet|  14343.0|754|      14.7|
|         ford|  15230.0|672|      13.1|
|       nissan|  11381.0|513|      10.0|
|       toyota|  13765.0|485|       9.5|
|        honda|  63834.0|306|       6.0|
|         jeep|  17487.0|262|       5.1|
|      hyundai|  11268.0|258|       5.0|
|          kia|  11085.0|219|       4.3|
|        dodge|  12916.0|187|       3.7|
|          gmc|  15903.0|192|       3.7|
|          ram|  11259.0|132|       2.6|
|          bmw|  20147.0|131|       2.6|
|       subaru|  14679.0|134|       2.6|
|   volkswagen|  11196.0|115|       2.2|
|        buick|  13444.0| 87|       1.7|
|         audi|  16524.0| 85|       1.7|
|mercedes-benz| 312339.0| 82|       1.6|
|        lexus|  15851.0| 76|       1.5|
|     cadillac|  17322.0| 75|       1.5|
|        mazda|  10337.0| 73|       1.4|
|     chrysler|  13169.0| 69|       1.3|
|        acura| 

In [23]:
#I will keep only manufacturers that have more than 5% of market share.

targetDF2 = targetDF\
.where((col("manufacturer") == "ford") |\
          (col("manufacturer") == "chevrolet") |\
          (col("manufacturer") == "honda") |\
          (col("manufacturer") == "jeep") |\
          (col("manufacturer") == "toyota") |\
          (col("manufacturer") == "hyundai") |\
          (col("manufacturer") == "nissan"))

In [24]:
targetDF2\
.groupBy("manufacturer")\
              .agg(round(avg("price"),0).alias("Avg Price"),round(avg('year'),0).alias("AVG Year"),\
                   count('manufacturer').alias("N."), round((count("manufacturer")/targetDF.count()*100),1).alias("% of total"))\
                 .orderBy("% of total",ascending=False).show(200)

print("It looks weired that Honda cars are 5x more expensive than ALL THE REST. Let's do some exploration.")

+------------+---------+--------+---+----------+
|manufacturer|Avg Price|AVG Year| N.|% of total|
+------------+---------+--------+---+----------+
|   chevrolet|  14343.0|  2017.0|754|      14.7|
|        ford|  15230.0|  2016.0|672|      13.1|
|      nissan|  11381.0|  2016.0|513|      10.0|
|      toyota|  13765.0|  2017.0|485|       9.5|
|       honda|  63834.0|  2016.0|306|       6.0|
|        jeep|  17487.0|  2016.0|262|       5.1|
|     hyundai|  11268.0|  2017.0|258|       5.0|
+------------+---------+--------+---+----------+

It looks weired that Honda cars are 5x more expensive than ALL THE REST. Let's do some exploration.


In [27]:
targetDF2.where(col("manufacturer") == "honda")\
.groupBy("model")\
              .agg(round(avg("price"),0).alias("Avg Price"),round(avg('year'),0).alias("AVG Year"),\
                   count('manufacturer').alias("N."), round((count("manufacturer")/targetDF.count()*100),1).alias("% of total"))\
                 .orderBy("Avg Price",ascending=False).show(10)

print("Clearly, I have a TERRIBLE outlie since I don't think a 2016 Honda Civic Sedan can cost 831K USD")

+--------------------+---------+--------+---+----------+
|               model|Avg Price|AVG Year| N.|% of total|
+--------------------+---------+--------+---+----------+
|         civic sedan| 831221.0|  2016.0| 19|       0.4|
|       pilot touring|  26963.0|  2016.0|  1|       0.0|
|          pilot ex-l|  26880.0|  2017.0|  1|       0.0|
|     ridgeline sport|  26500.0|  2017.0|  1|       0.0|
|           cr-v ex-l|  25650.0|  2017.0|  1|       0.0|
|             odyssey|  25000.0|  2016.0|  1|       0.0|
|clarity plug-in h...|  24999.0|  2018.0|  1|       0.0|
|civic sport hatch...|  23995.0|  2019.0|  1|       0.0|
|              crv ex|  21498.0|  2018.0|  3|       0.1|
|      cr-v ex-l grey|  21495.0|  2016.0|  1|       0.0|
+--------------------+---------+--------+---+----------+
only showing top 10 rows

Clearly, I have a TERRIBLE outlie since I don't think a 2016 Honda Civic Sedan can cost 831K USD


In [29]:
targetDF2.approxQuantile("price",[0.25],0)

[7970.0]

In [30]:
targetDF2.approxQuantile("price",[0.75],0)

[18900.0]

In [32]:
q1 = 7970
q3 = 18900

iqr = q3-q1 #Interquantile range
fence_low = q1-1.5*iqr
fence_high = q3+1.5*iqr


In [34]:
targetDF3 = targetDF2.where((col("price") > fence_low) & (col("price") < fence_high))

print("I have removed", targetDF2.count() - targetDF3.count(), "outliers.-")

I have removed 82 outliers.-


In [39]:
print("Before......")

targetDF2.select('price').summary().show()

print("After.....")

targetDF3.select('price').summary().show()

Before......
+-------+------------------+
|summary|             price|
+-------+------------------+
|  count|              3250|
|   mean|18641.657846153845|
| stddev|272722.38852896006|
|    min|                 0|
|    25%|              7970|
|    50%|             13600|
|    75%|             18900|
|    max|          15550499|
+-------+------------------+

After.....
+-------+------------------+
|summary|             price|
+-------+------------------+
|  count|              3168|
|   mean|13053.252525252525|
| stddev| 8742.648262886763|
|    min|                 0|
|    25%|              7700|
|    50%|             13450|
|    75%|             17997|
|    max|             35000|
+-------+------------------+



In [40]:
targetDF3.groupBy("manufacturer","model")\
              .agg(round(avg("price"),0).alias("Avg Price"),\
                   count('manufacturer').alias("N."), round((count("manufacturer")/targetDF.count()*100),1).alias("% of total"))\
                 .orderBy("Avg Price",ascending=False)\
.where(col("% of total") > 1).show(200)


print("Our final choice is the Ford Fusion.-")

+------------+--------------+---------+---+----------+
|manufacturer|         model|Avg Price| N.|% of total|
+------------+--------------+---------+---+----------+
|      toyota|        tacoma|  21649.0| 64|       1.2|
|   chevrolet|         tahoe|  14139.0| 60|       1.2|
|        ford|        escape|  12896.0| 65|       1.3|
|   chevrolet|silverado 1500|  11147.0| 90|       1.8|
|        ford|        fusion|  10621.0| 59|       1.2|
|      nissan|        altima|   9870.0| 67|       1.3|
|      toyota|         camry|   8985.0| 68|       1.3|
|        ford|         focus|   8780.0| 55|       1.1|
|      nissan|        sentra|   7796.0| 76|       1.5|
|      toyota|       corolla|   6942.0| 69|       1.3|
+------------+--------------+---------+---+----------+

Our final choice is the Ford Fusion
